# LX 496/796  Formalizing meaning, part II: SHRDLU

This time we'll finish this SHRLDU project, I think we've extracted as much as we can out of it at this point.  After this is finished, we'll have something that has the outlines of a project and we've had a chance to dive into knowledge representation, parsing, semantic representation, user interaction, graphical representation.  I hope in retrospect it will have seemed worth using it as a focal point for addressing these various different aspects of language in a computation context, but there are still many ways it could be extended.  It could be given a third dimension, it could be given more of a discourse memory (like the real SHRDLU, allowing it to learn names for configurations, resolve pronouns), it could be given a planner (so that it can work out a plan of action to get from the current state to the goal state, possibly involving several moves), maybe given a bigger vocabulary and more actions.  Lots of places it could go, but not necessarily usefull for our purposes.

But let's get it to that point, we're not there yet.  And this is actually a REALLY LONG notebook.  Here is what we have so far:

- representation of the objects in the world
- grammar for syntactic parsing and semantic composition
- display module to show the current state of the world

This time we will:

- supplement the semantic rules to allow for quantifiers
- gather user input and respond to it
- extend the grammar to add questions and imperatives


### Note about ACTIVITY Chat cells

Unlike previous notebooks, this one has some interactive elements, cells which make the robot start and wait for you to type something.  That means "run all" or "run before" won't work well if you work on this and the return to it later.  You might consider commenting out the `chat()` cells once you get past them, so that next time you pick up working on it you can just "run all" cells before the point where you stopped.  But at least be aware of this.  I think I have marked all of the cells that contain `chat()` in them with **ACTIVITY. Chat cell** in the table of contents so that at least you can jump to them.

> What I mean by "commenting out" is changing
>```
chat()
```
> to
>```
># chat()
```
>in order for Python to see it just as a comment/annotation rather than a command to execute.

## Setting up the world and visualization code

We'll start off by setting up the world.  This is the same as where we ended up after the previous homeworks.

We have 13 objects, 8 of which are
squares that represent the floor (4 of which are odd, 4 of which are even),
and 5 of which are shapes of various kinds (block, pyramid, ball) with various
properies (red, big, etc.).  Three of the objects are on the floor, one object
is on another one, and one is in the robot hand.

We will also just recreate the world visualization system and tree drawing functions we had earlier (developed in class), again without really any further comment, just copying in the code from earlier.  It will be a while before we use some of it, but it will be ready for us when we get there.

In [ ]:
# bring in the feature grammar library for parsing trees
import nltk
from nltk import grammar
# install svgling so we can illustrate things
!pip install svgling
import svgling
# define the domain of individuals
# also define the list of squares separately so that we can use it
squares = ['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8']
dom = {'a', 'b', 'c', 'd', 'e'} | set(squares)
# define the valuation function
# (specifies the state of the world, what things are red, what squares are odd)
valstr = """
square => {s1, s2, s3, s4, s5, s6, s7, s8}
odd => {s1, s3, s5, s7}
even => {s2, s4, s6, s8}
block => {a, b}
pyramid => {c, e}
ball => {d}
thing => {a, b, c, d, e}
red => {a}
blue => {b, e}
green => {c, d}
on => {(a,s1),(b,s2),(d,s4),(c,d)}
held => {e}
"""
val = nltk.sem.Valuation.fromstring(valstr)
m = nltk.Model(dom, val)
g = nltk.Assignment(dom)

# drawing the world
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import Wedge

# convert a feature tree into something svgling can draw
def convert_tree(node):
  nodetype = type(node).__name__
  if nodetype == 'Tree':
    keylist = list(node.label().keys())
    label = node.label()[keylist[0]]
    if 'SEM' in node.label():
      label += "\n" + str(node.label()['SEM'])
    return tuple([label] + [convert_tree(child) for child in node])
  else:
    return node

# functions for interpreting the world
def obj_in_hand():
  if len(val['held']) == 0:
    return None
  else:
    return list(val['held'])[0][0]

# slight change here in order to allow for our named individuals Cpyr and Dball
def obj_properties(obj):
    return {v for v in val if type(val[v]) is set and (obj,) in val[v]}

def obj_shape(obj):
  shape_properties = {'block', 'ball', 'pyramid', 'square'}
  shape_property = shape_properties & obj_properties(obj)
  if len(shape_property) == 0:
    return None
  else:
    return list(shape_property)[0]

def obj_color(obj):
  color_properties = {'red', 'blue', 'green'}
  color_property = color_properties & obj_properties(obj)
  if len(color_property) == 0:
    return None
  else:
    return list(color_property)[0]

def whats_on(obj):
  # ask: what is on the current support (obj)?
  f = nltk.sem.Expression.fromstring('on(x,s)')
  g2 = nltk.Assignment(dom, [ ('s', obj ) ] )
  try:
    next_obj = list(m.satisfiers(f, 'x', g2))[0]
  except:
    next_obj = None
  return next_obj

def build_stack(square):
  stack = [square]
  while True:
    next_obj = whats_on(stack[-1])
    if next_obj:
      stack.append(next_obj)
    else:
      break
  return stack

def draw_shape(obj, col, row):
  side = .9
  half = side / 2
  properties = obj_properties(obj)
  shape = obj_shape(obj)
  color = obj_color(obj)
  if shape == 'ball':
    patch = plt.Circle( (col+half, row+half), half, fc=color)
  elif shape == 'pyramid':
    patch = plt.Polygon( ((col, row), (col+half, row+side), (col+side, row)), fc=color)
  elif shape == 'square':
    if 'even' in properties:
      pattern = '//'
    else:
      pattern = '\\\\'
    patch = plt.Rectangle((col, row), side, side, fc='purple', ec='cyan', hatch=pattern)
  elif shape == 'block':
    patch = plt.Rectangle((col, row), side, side, fc=color)
  else: # hand
    patch = Wedge((col+1, row + half), half, 30, 330, fc='yellow', ec='black')
  plt.gca().add_patch(patch)
  plt.gca().annotate(obj, (col+half, row+half), va='center', ha='center', color='white', weight='bold', fontsize=16)

def plot_world():
  plt.axes()
  stacks = [build_stack(s) for s in squares]
  tallest = max( [len(s) for s in stacks] )
  if obj_in_hand():
    draw_shape(obj_in_hand(), 1, tallest+1)
  draw_shape(None, 0, tallest + 1)
  for col in range(8):
    stack = stacks[col]
    for row, obj in enumerate(stack):
      draw_shape(obj, col, row)
  plt.axis('scaled')
  plt.show()


## The architecture

In order to parse a sentence, we need
 - a model (domain and valuation function) and an assignment
 - a grammar (which specifies the semantics for words and non-terminal nodes)
 - a parser (which resolves the words into a structure)

In order to interact with the robot, we need
 - a place to type
 - evaluation of the sentence typed
 - specification of actions to take depending on the meaning of the sentence

**The goal:** We want to be able to say to "a green pyramid" or "every odd square" and have it figure out what object(s) are being referred to.  We want to be able to say "a ball is on an even square" (a declarative sentence) and have it evaluate whether it is true or not, and we want to be able to ask "is a table on an even square" (a yes-no question) and have it detect that it is a question and answer yes or no.  And we want to be able to say "put the green pyramid on an odd square" (an imperative) and have it adjust the world accordingly.


## Setting up the grammar

We're going to set up a basic grammar and semantics, along the lines of what we had in the previous part of the homework, and then we will extend it.

We will define a key-value dictionary called `gramspec` that will have as its keys the bit of the grammar the value defines.  The names of the keys are arbitrary, just a mneumonic for us to remember which part the value refers to.

> Note on the programming strategy: I have made a specific choice here about how we will build up the grammar, which is to create the grammar out of a bunch of little fragments.  Grammars do not *actually* need to be constructed this way, it can be all done in one big string.  From a kind of pedagogical perspective, I think doing it in little chunks is better, because it's a little easier to focus on each little part and talk about why we're doing what we're doing in that part.  It also makes it easier to replace just one part of the grammar while leaving the rest of it as it was.  But in the end, we will just smash all the little parts together into one big string and then make the grammar out of that.

We are presupposing a grammar that should construct trees like the following one.  We will start by having basically one "verb" which is the combination of a copula *be* and a preposition *on*, and is transitive.  So we need to design our grammar to parse these trees, and parse the semantic values of each node here.

It turns out that the noun phrases are going to be fairly complicated part (the determiners like "the" and "every" most of all), so let's try to build up the rest of this first.  In order to postpone the need to define NPs and the semantics within them, let's name a couple of the objects (Cpyr and Dball) so we can refer to them by name.

Anticipating that we are going to add questions and imperatives as clause types later, we will start the tree with CP, with the idea that the C head is where the information about clause type is stored.  In syntax, we would say that a declarative sentence has a silent C, but having silent elements would introduce a kind of complexity we don't want to confront right now.  So we can define CP such that it has a CBAR node (needed if we later add *wh*-questions) whose single daughter is TP in the situation where the actual C head would be silent.  That is, a declarative statement will have a CP that looks like the tree below.

> The tree makes distinctions we need here, but does not make all of the distinctions that theoretical syntax would.  There is no TBAR, no *vP* or anything like that.  If we start confronting how to parse a sentence that contains silent elements, we can start elaborating the tree to be more like syntactic theory indicates, but for these purposes we will just stick with a slimmed down tree that has only those things that we rely on.
>
> Also, nearly all of the trees being drawn in here are to illustrate points about what we're parsing and constructing. They are being constructed by hand by just listing the nodes.  These are not being computed by parsing anything.

In [ ]:
# draw a tree illustrating the structure our grammar is intended to build
svgling.draw_tree("CP", ("CBAR", ("TP", ("DP","Cpyr") , \
  ("VP", ("V","is"), ("PP", ("P","on"), ("DP","Dball"))))))

### Defining the names Cpyr and Dball

We'll start with the easy parts.  First we create the empty dictionary, and then we add an entry called `names`.  This is going to be used to name two specific objects.  We are going to name the green pyramid (c) "Cpyr" and the green ball (d) "Dball".  

> Each PSR (phrase structure rule) has the node label on the left, followed by a feature list enclosed in `[`square brackets`]`, with the `SEM` value indicating a semantically active value by putting the expression in `<`angled brackets`>`.  A single hypen is used to form the arrow pointing to what the node is rewritten as (also known as the daughters of that node).  On the right, if we have a string enclosed in `'`single quote marks`'` that is a word we expect to see in the sentence.  It is also of course possible to have nodes to the right of the arrow, as will be exemplified shortly.
>
> Also remember that the multi-line string is formed with `"""`triple quotation marks`"""` and the prepended `r` means "raw", which keeps Python from interpreting `\` as an escape character and instead just interprets it as a backslash.

In [ ]:
# create the empty grammar specification dictionary
gramspec = {}
# Add the specification for the name definition
gramspec['names'] = r"""
DP[SEM=<cpyr>] -> 'cpyr'
DP[SEM=<dball>] -> 'dball'
"""

We also need to update `val` so it knows what individuals "cpyr" and "dball" are. 

In [ ]:
val['cpyr'] = 'c'
val['dball'] = 'd'

### Defining the CP

Then we will define CP.  Remember that the `% start CP` line has to be up at the top of the overall grammar specification string, and it defines what the symbol is at the top of the tree.

> Also, it is probably worth taking a second to talk about the CP rules below themselves.  Remember how these rules work from the previous homework.  The parts in square brackets define features, and we are defining two features here.  One is `SEM` (for "SEMantics") and one is `CT` (for "Clause Type").  The `?cbar` and `?ct` and `?tp` are variables.  They name the values that appear on the right, and they are used to set values on the left.  So the first rule says "Let's call the `CT` value (from `CBAR`) `?ct` and the `SEM` value (from `CBAR`) `?cbar`.  We can build a `CP` out of a `CBAR` (this corresponds to a tree that has just a single branch from CP down to CBAR, no specifier), and the `SEM` value of `CP` will be `?cbar` (so, just the same as that value was for `CBAR`) and the `CT` value of `CP` will be `?ct` (so, just the same as that value was for `CBAR`).  Essentially, the `SEM` and `CT` features are being "passed up" from `CBAR` to `CP`.  That's what the first rule says.  The second rule says that the `SEM` and `CT` features are being passed up from `TP` to `CBAR` too.  So, this is a lot of nothing, semantically speaking, but we may be tweaking some of this later.

In [ ]:
# define CP as containing Cbar and TP, with trivial semantics
gramspec['cp'] = r"""
% start CP
CP[SEM=?cbar, CT=?ct] -> CBAR[SEM=?cbar, CT=?ct]
CBAR[SEM=?tp, CT=?ct] -> TP[SEM=?tp, CT=?ct]
"""

We can't build a *useful* grammar out of just these two fragments yet, but we can still see how we will be filling in more fragments until we have a workable grammar.  Note too that defining them like `gramspec['cp'] = ...` means that later, if/when we want to change just the CP part of the grammar, we can just update `gramspec['cp']` and it will be integrated into the grammar.  It seems cleaner, division and conquest.

We can define the following function to assemble the grammar specification string from the dictionary entries.

In [ ]:
def assemble_spec(gramspec):
  return "".join(sorted(gramspec.values()))

This just concatenates all the fragments defined in the dictionary.  Two notes:
 - `.values()` gives us just the values and not the keys from the dictionary.  So just the rules, not the label like 'cp' or 'name'.
 - `sorted()` is a sneaky trick because we need to get the `% start CP` up at the top, and the fragment that starts with `%` will be alphabetically sorted before anything else we have.
 
When we try it, we get:

In [ ]:
print(assemble_spec(gramspec))

To parse a sentence (recall), we need to construct a grammar, a parser, break the sentence into words, and parse it.  We can put that into a function that does all of that at once, and returns the list of parses.

In [ ]:
# parse a sentence to compute its semantic value(s)
# takes the gramspec (dictionary of grammar fragments) and a sentence (a single string)
def sent_parse(gramspec, sentence):
  full_spec = assemble_spec(gramspec)
  new_grammar = grammar.FeatureGrammar.fromstring(full_spec)
  parser = nltk.FeatureChartParser(new_grammar)
  words = sentence.split()
  return list(parser.parse(words))

We don't yet have a complete enough grammar to parse anything (and we won't for quite a while), so we can't test this out yet, but it's a little easier to put these function definitions up here before we start wading into the details of the grammar itself.

Another thing we will want to do is evaluate whether the semantics we get are true of our model of the world, so we can define a function that does that now as well.  Because there might be several parses in a list (notice that `sent_parse` returns a list of parses), this will evaluate all of the semantic expressions in a list.

In [ ]:
# evaluate whether a list of expressions are true in the model
def eval_sent(m, g, expressions):
  return [m.satisfy(expr, g) for expr in expressions]

This one we can test if we make our own expression by hand.  We can explicitly create an expression using `nltk.Expression.fromstring()`.

In [ ]:
# test eval_sent out on 'there is a green pyramid' and 'there is a red pyramid'
# first build the two expressions
# expr1 = there is a green pyramid
expr1 = nltk.Expression.fromstring('exists x.(green(x) & pyramid(x))')
# expr2 = there is a red pyramid
expr2 = nltk.Expression.fromstring('exists x.(red(x) & pyramid(x))')
# execute eval_sent with respect to the model, providing a list of these two expressions
eval_sent(m, g, [expr1, expr2])

And, lastly for the moment, to mediate between `sent_parse` and `eval_sent` we can define `sent_sem` that will construct a list of parses and extract the `SEM` feature from the top of them (the semantic representation of the entire sentence) to make expressions (which we can then give to `eval_sent` to evaluate against the model).  This doesn't really do very much, it just will make what we do later marginally more readable.

In [ ]:
# get a list of the parses of a sentence, then
# return only the overall semantics (of top node) of each parse, in a list
def sent_sem(gramspec, sent):
  parses = sent_parse(gramspec, sent)
  return [x.label()['SEM'] for x in parses]

Again, we can't test this yet because we can't parse anything yet, but it should be clear that `sent_sem()` collects the parses (using `sent_parse()`) and then gets the `SEM` feature from the top of each parse (the `label()` of the parse).

To see what fragments have been added to the specification dictionary so far, you can submit it to `list()`.  This will give you a list of just the keys.  (And if you want just the values without the keys, you can use the `values()` method, like we saw in the definition of `assemble_spec` above.)  Since we have only defined the grammar fragments called "cp" and "names" so far, those are the keys in the list.

In [ ]:
# display the keys in the dictionary (the names of the fragments) we have so far
list(gramspec)

### Defining the VP and PP (the predicates)

The main kind of sentence we will have (at least until we get to the imperative verbs) we be "be" plus a PP (like "on the block").  And the "be" doesn't even contribute anything to the semantics.  The real predicate in "a pyramid is on the block" is "on".  So, we'll define a VP that has "is" as its verb, and adds no meaning at all, just passes up the meaning of the PP contained within it.

### TASK 1. Define the PSR for VP

Replace the blank line below with the appropriate PSR rule that says that VP has daughters VCOP and PP, and the SEM value of VP is the same as the SEM value of its PP daughter.

> You can look at how the CP was defined for hints on how to do this kind of trivial semantics.  But note that the CP passes up both SEM and CT features, but here the VP only needs to be concerned with SEM features not CT features. It will be a little while yet before we can test to see if this worked right.

In [ ]:
# Task 1. Define VP as being "is" plus a PP, with the same semantics as the PP
gramspec['vp'] = r"""

VCOP -> 'is'
"""

And then define PP as being combined of the semantics of the P (like "on") and the semantics of the DP object.  As a reminder, the way you read this is that the `?p` on the right side and the `?dp` on the right side are variable labels, and then we use those labels on the left side.  So this is saying: Call the semantics of the P `?p` and call the semantics of the DP `?dp`.  The semantics of the combination (PP) is going to be what you get when you apply `?p` to `?dp`.

In [ ]:
# define PP as applying the function P to the argument DP
gramspec['pp'] = r"""
PP[SEM=<?p(?dp)>] -> P[SEM=?p] DP[SEM=?dp]
"""

We then need to define what the Ps are.  We have one P relation defined in the model of the world, "on".  "On" is a two-place predicate, it is a relation between two individual objects.  To determine whether "on" gives us true or false, we need to get two individuals (call them `x` and `y`), and then check to see if `(x,y)` is in the definition of `on` in the model.

> Breaking this down, we want "on dball" to be true of anything `y` such that `(y,d)` is in the list of pairs that define `on`.  So the `SEM` value of that PP should be:
>
>```
\y.on(y,d)
```
>
>...which is true of any things that are on `d` (dball).  Now, looking inside the PP, it is made of "on" and "dball", and we presumably want to get the `d` part there from "dball", so we want to factor that out of the definition of the P "on".  That is, we want the P "on" to be something that, if given an `x` (like "dball"), it will yield the function above.
>
>```
\x.\y.on(y,x)
```

In [ ]:
# define the two-place predicate 'on'
gramspec['p'] = r"""
P[SEM=<\x.\y.on(y,x)>] -> 'on'
"""

### Defining the TP

And then, to complete a (comically simple) grammar, we can define the part at the top of the tree that combines the subject and the verb.
We already defined CP up at the beginning, but we also need to define TP.

The definition of TP is (for now) going to take the function provided by the VP and apply it to the subject DP.  This is basically familiar from last time.

> NOTE: We have defined the TP here as having the clause type `decl`.  It does not get this from either of its constituent parts, the TP is the origin of the `CT` feature.  But if you look at the rules for CBAR and CP, they will just pass that feature up, so that at the top the CP will have the `CT` feature that TP set.

In [ ]:
# define TP as taking the VP predicate as a function applied to the DP subject as its argument
gramspec['tp'] = r"""
TP[SEM=<?vp(?subj)>, CT='decl'] -> DP[SEM=?subj] VP[SEM=?vp]
"""

We now have a grammar just barely sufficient to parse a couple of sentences.  We can parse "cpyr is on dball" and "dball is on cpyr" if everything worked ok.  But, sure, it's a start.  Let's try it out.  First, we'll have a look at our grammar specification so far, and then get the parses.

In [ ]:
# look at our grammar, assembled from fragments
print(assemble_spec(gramspec))

In [ ]:
# get a list of semantic values for parses for 'cpyr is on dball', print them, and evaluate them
parse_sems = sent_sem(gramspec, 'cpyr is on dball')
print(parse_sems)
print(eval_sent(m, g, parse_sems))

> **Task 1 check:** This is the first place where we can really see if your Task 1 worked.  If you see the following thing above after running the previous cell, then your Task 1 answer worked. Otherwise, there is probably something wrong with your Task 1 answer.
> ```
[<ApplicationExpression on(cpyr,dball)>]
[True]
```


### TASK 2. Dball is not on Cpyr

Do what we just did above, except showing instead that it is not true that dball is on cpyr.

In [ ]:
# Task 2.  Show that 'dball is on cpyr' is false in the model


Ok, not bad.  We now have a grammar and semantics that can handle our two named individuals and the "on" relation.  It can parse sentences.  We are on our way.

## Handling subject quantifiers

Back to the block world and our ever-growing grammar.
How do we handle "a block" or "every pyramid"?  We'll return to this shortly, but briefly: "A pyramid is on dball" asserts that the intersection of the set of pyramids and the set of things on dball is nonempty.  That there is some individual in the model of the world that has both properties of being a pyramid and of being on dball. A quantifier is taking two properties and making an expression out of them.

We'll start somewhat small and easy and define the properties first.  What is the meaning of the property (or "predicate") *pyramid*?  In the present context, *x* is a pyramid if *x* is in the set of pyramids.  We defined the set of pyramids back when we defined the valuation function. The expression evaluator will understand how `pyramid(y)` relates to the `pyramid` set we defined, and will return true when `y` is in that `pyramid` set. So, the expression defining the word "pyramid" is just `\y.pyramid(y)`.

To see how to define the semantics for "a" and the DP combining "a" and "pyramid", let's look at the combination structure we are aiming for, relating to that part of the parse tree.


In [ ]:
# draw a tree illustrating the semantic combination for a quantified subject
svgling.draw_tree("TP\nexists x.(pyramid(x) & on(x,d))", \
  ("DP", ("Det", "a"), ("NP\n\\y.pyramid(y)", "pyramid")), \
  ("VP\n\\y.on(y,d)", "is on dball"))

The semantic value of "is on dball" is `\y.on(y,d)` -- we can just assume that for now without deriving it from its components.  And the semantic value of the whole sentence is `exists x.(pyramid(x) & on(x,d))` -- that is, there is an *x* that is both a pyramid and on dball.  Clearly we want to get the pyramid part from the NP and the being on dball part from the VP.  Each is a predicate.  So, we want the DP to take a predicate (is on dball) and return the formula at the top.  Thus, the DP should look like this:

In [ ]:
# draw a tree illustrating the semantic combination for a quantified subject
svgling.draw_tree("TP\nexists x.(pyramid(x) & on(x,d))", \
  ("DP\n\\Q exists x.(pyramid(x) & Q(x))", \
  ("Det", "a"), ("NP\n\\y.pyramid(y)", "pyramid")), \
  ("VP\n\\y.on(y,d)", "is on dball"))

And then that means that the Det should take a predicate (pyramid) and return the value we just worked out for DP.

In [ ]:
# draw a tree illustrating the semantic combination for a quantified subject
svgling.draw_tree("TP\nexists x.(pyramid(x) & on(x,d))", \
  ("DP\n\\Q exists x.(pyramid(x) & Q(x))", \
  ("Det\n\\P\\Q exists x.(P(x) & Q(x))", "a"), \
  ("NP\n\\y.pyramid(y)", "pyramid")), ("VP\n\\y.on(y,d)", "is on dball"))

So let's add this to the grammar.  The logical notation used above in the tree is what seems clearest to me. But the NLTK expressions are a little bit different, in that they omit the second lambda.  If you compare the definition of the Det "a" below you will see that there is an implicit lambda between the P and Q.

In [ ]:
# define pyramid
gramspec['np'] = r"""
NP[SEM=<\y.pyramid(y)>] -> 'pyramid'
"""

# define the determiner "a"
gramspec['det'] = r"""
D[SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'a'
"""

# define DP that combines them;
# the Det is a function that takes the NP as an argument
gramspec['dp'] = r"""
DP[SEM=<?det(?np)>] -> D[SEM=?det] NP[SEM=?np]
"""

One remaining issue is that the existing TP rule says that the VP is a function that takes the subject DP as the argument.   In this case, we need it to go the other way.  For a subject quantifier, the subject DP is a function that takes the predicate VP as an argument.

So, we redefine TP so that it applies the subject function to the VP argument.

In [ ]:
# redefine TP for quantified subjects
gramspec['tp'] = r"""
TP[SEM=<?subj(?vp)>, CT='decl'] -> DP[SEM=?subj] VP[SEM=?vp]
"""

Although we probably do not really need to keep "cpyr" and "dball" (our green pyramid and ball) as proper names,
they are currently in our grammar, and we just removed the ability for them to appear as subjects
(because they are defined as individuals and not functions).  So, we can redefine them as functions
for the moment just so the grammar remains consistent.

The definitions that do this are given below.  The one for Cpyr says that "cpyr" is true of all of the properties that hold of Cpyr.  It's kind of a fancy way of saying "the set of Cpyr's properties."  This is now identifying individuals in terms of their properties rather than by individual reference.  But it makes this the same semantic type as a quantifier subject, and so it allow us to have either names or quantifiers as subjects.  And it's basically equivalent to say "the set of swimmers includes Pat" (the old way, meaning that the property of being a swimmer holds of Pat), or (in the new way) "the set of Pat's properties includes being a swimmer".

In [ ]:
# redefine the names to be functions of properties
gramspec['names'] = r"""
DP[SEM=<\P.P(cpyr)>] -> 'cpyr'
DP[SEM=<\P.P(dball)>] -> 'dball'
"""

The way it would combine looks like the tree below.

In [ ]:
# draw a tree illustrating the semantic combination for a higher type name subject
svgling.draw_tree("TP\n(\\y.on(y,d))(c)\non(c,d)", ("DP\n\\Q.Q(c)", "cpyr"), \
  ("VP\n\\y.on(y,d)", "is on dball"))

It's worth pausing for a second on that combination at the top.  The way it is to be understood is that the formula under TP, `(\y.on(y,d))(c)`, is the semantic value of the top node, and the formula under that, `on(c,d)`, is equivalent, reduced/simplified by applying the function to the argument. The notation is intended to mean that `\y.on(y,d)` is a function and that it is being given `c` as its argument.  We can apply that function to that argument, as I do in the third line, "reducing it" to `on(c,d)` by taking the argument (`c`) and replacing it in the result for `y`.  This arises from the DP (which requires a property it will refer to as `Q`) that is then applied to `c`.  The VP represents the property that the DP takes as an argument.

> To take a much simpler example, if that was hard to follow: suppose we have a function we call `f` that is defined as `\x.2 * x` (which multiplies a number by 2).  `f(3)` is equivalently `(\x.2 * x)(3)`, and if you take the argument (`3`) and substitute it in for `x`, you can simplify/reduce that to just `2 * 3`.  That essentially what is being described just above.


## Handling object quantifiers

The last thing we need before we can finally start parsing real things is a way to have a quantifier in object position.  All of our DPs are now a function taking a property and returning a truth value.  That is exactly what we want in the subject position, but it is *not* obviously what we want in object position.  We are going to need to redefine *on* so that instead of taking an individual *x* and returning the predicate "is on *x*", it instead takes a set of properties and returns a predicate something like "is on something with a property in this set".


In [ ]:
# draw a tree illustrating the semantic combination for a quantified object of on
svgling.draw_tree("PP\n\\x.(on(x,dball))", ("P", "on"), ("DP\n\\P.P(dball)"))

Let's try to work out what we need here.  It is hard to grasp.  I attempted to describe this in class with near zero success.  But maybe it can be made to work here in a Jupyter notebook document.

We want *on* to be a function that takes the DP as an argument.  So we know that we start with something like:

`\F` ...

Where `F` here represents a function of predicates, and in this particular case is a function that is true of the predicates that hold of Dball.  Among the things that might be true of Dball is that certain things might be on it.  Cpyr might be on it, etc.  In such a case, `on(c,d)` would be true, which means that the predicate `\x.on(c,x)` ("Cpyr is on it") would be one of the predicates holding of Dball.  What we want to do now is make a predicate that is true of all the things that are on Dball (and false of the other things).  To do this, we can imagine going through all the things in the domain, referring to the current thing as `y` and checking whether `\x.on(y,x)` is among the predicates that are true of Dball (that is, if `y` is on Dball).

So if `F` is true of properties of Dball, we can check, for each thing `y` in the domain, whether `F(\x.on(y,x))` is true.  That is whether "`y` is on Dball" is a true property of Dball.  Then it is just a matter of collecting all those `y`s that are on Dball.

So, to spell it all the way out,

| formula | prose |
|---|---|
| `\F` ... | given a predicate of properties, called `F` |
| ... `\x` ... | given an individual, called `x` |
| ... `F(\y.on(x,y))` | true if `x` being on it is one of the properties of `F` |

That may be about as clear as I can make it. It *does* make sense, I promise.  Requires a fair amount of mental gymnastics to get there, but that is why we can define *on* as below.  The good thing is, having made it work for the conceptually simpler case of a name, it will automatically also generalize to quantified objects.

In [ ]:
# redefine the P "on" to allow for quantified objects
gramspec['p'] = r"""
P[SEM=<\F x.F(\y.on(x,y))>] -> 'on'
"""

And now we can stand back and admire our work.

In [ ]:
print(assemble_spec(gramspec))

Does this do what we want?  The goal is to parse "a pyramid is on dball" into `exists x.(pyramid(x) & on(x,d))`, and then be able to check it against the model.

In [ ]:
parse_sems = sent_sem(gramspec, 'a pyramid is on dball')
print(parse_sems)
print(eval_sent(m, g, parse_sems))

Success. Sweet.

In [ ]:
print(eval_sent(m, g, sent_sem(gramspec, "a pyramid is on cpyr")))

With that behind us, we can go ahead and define the grammar and semantics for all of the rest of the nouns.  They are "block", "ball", "square", and "thing" (and "pyramid").
All of the rules are going to be the same form as the rule for "pyramid" was.
So, we can be a little bit clever and save ourselves some typing (and potential typos) by building the multi-line string programmatically.
Specifically, we'll iterate over a list of the nouns, and make a rule for each one from a template.  Like so:

In [ ]:
# build the grammar fragment for the nouns
nouns = ["block", "ball", "square", "pyramid", "thing"]
nounstrings = ["NP[SEM=<\\x.{n}(x)>] -> '{n}'\n".format(n=noun) for noun in nouns]
gramspec['np'] = "".join(nounstrings)
# redefine the 'np' portion of the grammar specification to have all the nouns
print(gramspec['np'])

## Adjectives

How about adjectives like "green" in "a green pyramid"?  So far, there is no place in the grammar for these.

Although we are again limiting ourselves to a pretty small domain, we can for now at least presume that
the adjectives we care about are "intersective".  So a green pyramid is both green and a pyramid.

> This is in contrast to something like "fake" -- it is not the case that a fake gun is both
> fake and a gun.

It is also possible to have more than one adjective
(like in "a big green pyramid").  So let's suppose that means we can in principle have
unboundedly many adjectives, and create a recursive rule that allows us to make an NP
out of an adjective and an NP.

```
NP[SEM=<...?...>] -> Adj[SEM=?adj] NP[SEM=?np]
```

So, what is the semantic value of the combination?  Well, both an NP and an Adj seem like
the same kind of property, a one-place predicate.  "Pyramid" is true of things that are
pyramids, "green" is true of things that are green.  And "green pyramid" is true of things
that are both green and pyramids.

In [ ]:
# define NP -> Adj NP
gramspec['adjnp'] = r"""
NP[SEM=<\x.(?adj(x) & ?np(x))>] -> Adj[SEM=?adj] NP[SEM=?np]
"""

### TASK 3. Define the adjectives

We can define all the adjectives, using the same kind of cleverness deployed in defining the nouns, using a list and a template to generate a string.  Fill in the rest of the `adjstrings = ` line on the model of how the nouns were defined.


In [ ]:
# Task 3. Build the grammar fragment for the adjectives
# build the grammar fragment for the adjectives
adjs = ["odd", "even", "red", "blue", "green"]
adjstrings = ["Adj[SEM=<\\x.{a}(x)>] -> '{a}'\n".format(a=adj) for adj in adjs]
gramspec['adj'] = "".join(adjstrings)
print(gramspec['adj'])

Did it work? Let's see if a green pyramid is on Dball.  And if a red pyramid is on Dball.  And if a green green green pyramid is on Dball.  Because we can.  The answers should be true, false, and true.

In [ ]:
print(eval_sent(m, g, sent_sem(gramspec, "a green pyramid is on dball")))
print(eval_sent(m, g, sent_sem(gramspec, "a red pyramid is on dball")))
print(eval_sent(m, g, sent_sem(gramspec, "a green green green pyramid is on dball")))

### TASK 4. On whether Cpyr is on a green ball and whether a pyramid is on a ball

Use the same technique as above to determine whether a) Cpyr is on a green ball.  And b) whether a pyramid is on a ball.

In [ ]:
# Task 4.  Show whether cpyr is on a green ball and whether a pyramid is on a ball


## More determiners

Let's also define a couple more determiners.  The semantics of "the" and "a" and "an" are all pretty much the same
for now at least.  "The" is definite, and we might want to make use of that at some later point, but for now we'll just treat them the same.

So, we want to redefine the determiners.  But, wait, where were they again?
We can see what fragments we've defined in `gramspec` by looking at `list(gramspec)`. 

In [ ]:
print(list(gramspec))

Ok, it is `gramspec['det']` that we want to redefine.  Let's remind ourselves of what was there before.

In [ ]:
print(gramspec['det'])

And then we can redefine it as below.  Take a look at the definition for "every" down there and see if you get it.  The `->` means "implies" or "entails".
If every dog swims, that means that being a dog entails swimming (although *not* being a dog doesn't have any implications wrt swimming).  The formalism says it takes a predicate (like "dog") and calls it `N`, and another predicate (like "swims") and calls it `Q`, and then the expression for the combined meaning is that for all `x`, if `N(x)` is true then `Q(x)` must also be true.

In [ ]:
# define more determiners
gramspec['det'] = r"""
D[SEM=<\N Q.all x.(N(x) -> Q(x))>] -> 'every'
D[SEM=<\N Q.exists x.(N(x) & Q(x))>] -> 'a'
D[SEM=<\N Q.exists x.(N(x) & Q(x))>] -> 'an'
D[SEM=<\N Q.exists x.(N(x) & Q(x))>] -> 'the'
"""

## Testing the parsing of declaratives with quantifiers

Now just to verify that it worked.  We are getting to the point where we should be able to parse pretty sophisticated sentences.

In [ ]:
print(eval_sent(m, g, sent_sem(gramspec, "every pyramid is on a ball")))
print(eval_sent(m, g, sent_sem(gramspec, "every green pyramid is on a ball")))
print(eval_sent(m, g, sent_sem(gramspec, "every block is on a square")))
print(eval_sent(m, g, sent_sem(gramspec, "a pyramid is on the ball")))
print(eval_sent(m, g, sent_sem(gramspec, "a green pyramid is on every ball")))
print(sent_parse(gramspec, "a green pyramid is on every ball")[0].label()['SEM'])

### TASK 5. Get the CT feature of a parsed sentence

We're about to start adding the ability to talk to the robot, and adding questions and imperatives as clause types. The robot is going to need to know what the clause type of a sentence is.

On the model of the last command above, extract the clause type feature of "a green pyramid is on every ball".  The model showed you how to extract the SEM feature.  The clause type is recorded in a parallel feature called CT.  The answer you get should be "`decl`".

In [ ]:
# Task 5.  Retrieve the CT feature of "a green pyramid is on every ball"


## Talking to the robot ##

Let's add some interactivity, now that the world and parser are set up.
The main thing this is going to do at first is check the truth of a sentence we give it.
So, let's first define something that will check whether a sentence is true.
It takes some parses
of a sentence and checks whether the truth conditions of the first parse
are met.

In [ ]:
def check_truth(parse):
  treesem = parse.label()['SEM']
  return m.satisfy(treesem, g)

Before we define the `chat()` function itself, we will consider how we want it to work.  In particularly we want it to ask for a sentence, parse it, and act.  One basic distinction it will make is in whether it has been asked a question (in which case it will answer), or has been provided with a declarative (in which case it will evaluate the truth of the statement), or has given an imperative (in which case it will modify the world as instructed).

In order to make the `chat()` function easier to extend in stages, I have again chosen to use an architecture for it that divides the problem into subcomponents.  The plan will be to define three different functions, one for each clause type we will encounter.  We will be starting with `do_declarative()` and this will be called upon whenever the sentence that `chat()` parses is clause type `decl`.

To make this even more modular, I will define a dictionary of "handlers", where the key is a clause type and the value is the function we call if that clause type is encountered.  That way, `chat()` doesn't need to know what possible clause types there are, it just finds the clause type, looks up what handler to use, and then uses it.  This will allow us to start with just the declarative type, and then add questions and imperatives as we go.

So the first thing I'm going to do is define `do_declarative()` and this handler dictionary that associates `do_declarative()` with the clause type `decl`.  What `do_declarative()` will do is, given a parse, evaluate whether it is true and report back, using the `check_truth()` function we just defined.

In [ ]:
def do_declarative(parse):
  if check_truth(parse):
    print('That is true.')
  else:
    print('That is not true.')
    
type_handlers = {'decl': do_declarative}

And now we can define the function that actually allows us to talk to the
robot.

> In the definition of `chat()` there is a `try:` block and an `except:` block.  This means that it will attempt to run the commands in the `try:` block, but it is anticipating that it might get an error.  Instead of stopping on the error, it will instead run the `except:` block when an error is encountered.  This is specifically in order to handle the case where you enter a word it doesn't know.  This would normally stop the program with an error, but instead what the robot will do is just press on by telling you it didn't understand and going back to get more input.


In [ ]:
def chat():
  print('Type bye to leave, type look to show the scene.')
  print()
  while True:
    sent = input("> ").lower()
    if sent == 'bye':
      break
    elif sent == 'look':
      plot_world()
      continue
    try:
      parses = sent_parse(gramspec, sent)
      treetype = parses[0].label()['CT']
      if treetype in type_handlers:
          type_handlers[treetype](parses[0])
    except Exception as e:
      print("Error: {}".format(e))
      print("Sorry, what?")
  print("Bye!")

Take a moment to try to understand what it is doing and then give
it a spin.

There are two "magic" statements you can say to the robot.  One is
'bye', which will end it (`break` exits the `while True` loop);
the other is 'look', which will draw the world and then go back and
get more input (`continue` goes back up and starts the `while True`
loop again).  Otherwise you can just give it sentences like "a blue block is on an even square" and it will report whether or not the sentence is true.

If it gets an error while parsing (for example, if you use a word
it does not know), then it will print "Sorry, what?" and get more input.

### ACTIVITY. Chat cell.

In [ ]:
chat()

## Questioning the world ##

It is a bit unnatural to just say things and have the robot confirm
whether each is or is not true.  It would be nicer if we could ask it.

All we need for yes-no questions is to fix up the parser so that it
understands when a yes-no question is asked.  The basic task the
program performs once the semantics is established is identical to
checking the truth of a statement (except it will say "Yes" instead
of "That is true").

The form of a yes-no question, in this limited grammar, is just "is DP PP".
Specifically, the auxiliary "is" appears at the front instead of between
the DP and PP.

The tree below will be the structure we will assume.  In a question, the VCOP is the first daughter of CBAR (putting it before the subject, representing movement of the auxiliary to C), and the TP is one that has had its auxiliary removed (TPNOAUX).  A TPNOAUX differs from a TP in that it has a VPNOAUX in it rather than having a VP in it. And a VPNOAUX differs from a VP in that it has no head VCOP, because the VCOP has already been moved up to C.  So VPNOAUX has only the PP within it.

In [ ]:
# draw a tree illustrating the structure our grammar is intended to build for ynq
svgling.draw_tree("CP", ("CBAR", ("VCOP", "is"), ("TPNOAUX", ("DP","Cpyr") , \
  ("VPNOAUX", ("PP", ("P","on"), ("DP","Dball"))))))

In [ ]:
# ynq
gramspec['ynq'] = r"""
CBAR[SEM=?tp, CT=?ct] -> VCOP TPNOAUX[SEM=?tp, CT=?ct]
TPNOAUX[SEM=<?subj(?vp)>, CT='ynq'] -> DP[SEM=?subj] VPNOAUX[SEM=?vp]
VPNOAUX[SEM=?pp] -> PP[SEM=?pp]
"""

Here, we defined a version of CBAR that has the VCOP first, and an
"TP-without-an-aux" (TPNOAUX) that has just a DP and a "VP-without-an-aux"
(VPNOAUX), which itself contains just a PP.

Crucially, the TPNOAUX rule sets the CT feature to `ynq` (and the CBAR rule including TPNOAUX passes that feature up to CBAR, and the CP rule from before will pass that up from CBAR).  So these will get the clause type `ynq`.

To add the ability to handle yes-no questions to chat, we define a `ynq` handler.  Here is where we see why making this so modular was better, it's really super easy to add yes-no question ability to our robot.

In [ ]:
# define the ynq handler

def do_ynq(parse):
  if check_truth(parse):
    print('Yes.')
  else:
    print('No.')

# add the handler to the list of things chat() handles
type_handlers['ynq'] = do_ynq

### ACTIVITY. Chat cell: is a red block on an odd square

Show that it worked by asking it
"is a red block on an odd square".  How about an even square?  Play around a little bit more.

*Note:* **You should not put a question mark at the end**, the grammar will not know
what to do with punctuation.

> This doesn't count as a "task" because the chat has no history, so there's no reliable record of whether it worked in what you submit.
> But just ask the robot anyway and see if it says the right thing.


In [ ]:
chat()

## Changing the world ##

After kind of a slow start, we've made a bunch of progress pretty quickly.
We now have a world, and we can state and ask things about it, and the robot
understands to the extent that it can determine the truth of statements based
on the facts of the world.

The last major thing we'll do is add the ability to change the configuration of
the world, by adding imperatives to the parser.  This is also where we have to
add a lot more "smarts" to the system, because although NLTK was able to take
care of the heavy lifting in the domain of parsing and evaluation of logical
formulae, we need to tell the robot how to actually affect the world.

The first thing we need to do is revise the grammar to have one more clause
type, an imperative.  An imperative has a silent (implicit) subject, but
is otherwise mostly the same as a statement.  We are also going to add two verbs, "take" and "put."  "Take" is a transitive verb that takes a DP object, and "put" is a ditransitive verb (VDT) that takes a DP object and a PP goal.  The trees below illustrate the structures we are defining.

In [ ]:
# draw a tree illustrating the intended structure for imperative take
svgling.draw_tree("CP", ("CBAR", ("TP", ("VP", ("V", "take"), ("DP", "Cpyr")))))

In [ ]:
# draw a tree illustrating the intended structure for imperative put
svgling.draw_tree("CP", ("CBAR", ("TP", ("VP", \
  ("VDT", "put"), ("DP", "Cpyr"), ("PP", ("P","on"), ("DP","Dball"))))))

In [ ]:
# imperative TP

gramspec['imp'] = r"""
TP[SEM=<?vp(hand)>, CT='imp'] -> VP[SEM=?vp]
"""

As you can
see, what it essentially does is looks for a sentence with no subject, and
assumes that "hand" (the robot's actor) is the subject, and sets the clause
type to 'imp'.  (If the sentence has a subject, then it will use the rule for TP that has a subject, and the clause type would be "decl".)

Now we add two verbs: *take* and *put*.  What we want the
robot to do if we tell it to *take* something is to put it in its hand, and
if we tell it to *put* something somewhere, it will do that (after first taking it into its hand, possibly after first putting something else down).

Although we'll go ahead and fill in the semantics here, we actually are
not going to wind up using a lot of the semantics it computes.  But, here
is a little more to add.  We are adding a transitive verb "take"
(which is very much like the transitive P "on") and a ditransitive
verb "put" that has both a DP and PP argument. 

In [ ]:
gramspec['acts'] = r"""
VP[SEM=<?obj(?v)>] -> V[SEM=?v] DP[SEM=?obj]
V[SEM=<\x y.take(y,x)>] -> 'take'
VP[SEM=<?v(?obj,?pp)>] -> VDT[SEM=?v] DP[SEM=?obj] PP[SEM=?pp]
VDT[SEM=<\Y X x.X(\z.Y(\y.put(x,y,z)))>] -> 'put'
"""

At this point it should already be able to parse "put a red block on an even square".  Let's try.

In [ ]:
# try parsing put a red block on an even square
print(sent_sem(gramspec, "put a red block on an even square")[0])
print(sent_parse(gramspec, "put a red block on an even square")[0].label()['CT'])

Great.  Succesful parse!
Though the robot doesn't know how to make the command happen.

And this is where things get a little complicateder, because there are a lot
of things to take into account.  We are going to be building a `do_imperative`
function like the `do_declarative` and `do_ynq` functions that will take the parse
and deal with it.

The `do_imperative` function is supposed to look at the imperative
tree, determine what verb was used (*take* or *put*), and then take the
action.  We're going start with *take* because it's simpler.


## Taking things

Think first about what the robot is supposed to do if it is asked to
take something.  If you say "take a block" then there are several
possible choices of a block to take, in principle.  The robot can just pick one.  However, you can't
take something that's underneath something else, so it needs to check for
that.  Also, taking something means putting it in its hand, but if it
is already holding something, it needs to put that thing down first.

Let's try a couple of things by hand before we try to automate anything.
First, we will work out the sentence "take a red block" by hand:

In [ ]:
parses = sent_parse(gramspec, "take a red block")
print(parses[0].label()['CT']) # should say 'imp'
print(parses[0]) # the cp
print()
print(parses[0][0]) # the cbar
print()
print(parses[0][0][0]) # the tp

As you can see, we're kind of working our way down the tree.  Adding one more `[0]` (referring to the leftmost daughter) will give us the VP, which we will name `vp` so that we can type fewer zeros:

In [ ]:
vp = parses[0][0][0][0]
print(vp) # the vp
print()
print(vp[0]) # the v
print()
print(vp[0][0]) # the word in the sentence

After a whole lot of `[0]`s, we managed to zoom down onto the actual verb.
Because we have a very limited grammar, this is reliable enough.  That is,
we can count on being able to figure out what verb we have by looking at
`vp[0][0][0]`.  So, in particular, we can check to see if it is "take".

> ...or, equivalently, `parses[0][0][0][0][0][0]` but, that makes the code pretty hard to understand.

Likewise, we can get the object (the thing we're asking the robot to take)
with this:

In [ ]:
obj = vp[1]
print(obj)

For the object, we actually do need to use the semantics NLTK computed
for us.  Specifically, we need to figure out what individuals in the
model the object can describe (so we know what the robot is choosing
between when it takes something).

The logical formula for
the object is:

In [ ]:
# get the SEM value of the object as an expression
obj_sem = obj.label()['SEM']
print(obj_sem) # \Q.exists x.(red(x) & block(x) & Q(x))

This is the right form for the subject of a sentence, it takes a
predicate (`Q`) and is true if there is an `x` such that `x` is red,
a block, and `Q` is true of it.  But our goal right now is not to
evaluate a sentence, but to find out what the objects are that this
DP can represent.  We want to know what the red blocks are.

After some reflection, the simplest thing I could come up with is
to make `Q` be kind of predicate that means "is y" and then use `satisfiers()` to find get a list of the values of `y` that work.  That is, what we're going to be asking is: What are the values of y
that make "there is a red block that is y" true?  It's a little
bit roundabout.  (If you see a better way to do this, let me know!)

To implement this, we will define a predicate I'll call
`isy`:

In [ ]:
isy = nltk.sem.Expression.fromstring(r'\x.(x=y)')

This is true for any individual that is... whatever y is pointing to.
So, if we substitute `isy` in for `Q`:

In [ ]:
this_obj = nltk.sem.ApplicationExpression(obj_sem, isy).simplify()
print(this_obj) # exists x.(red(x) & block(x) & (x = y))

We now have an open formula (on `y`) that we can check for satisfiers on.
As I indicated, this is wandering around in the weeds a bit, but the main
thing is that we can get a list of the individuals that are red blocks by doing this:

In [ ]:
options = m.satisfiers(this_obj, 'y', g)
print(options) # {'a'}

Having walked through that process by hand, let's just commit that to a function that will
do all of that and give us a set of options. 

In [ ]:
def find_options(obj):
  obj_sem = obj.label()['SEM']
  isy = nltk.sem.Expression.fromstring(r'\x.(x=y)')
  this_obj = nltk.sem.ApplicationExpression(obj_sem, isy).simplify()
  options = m.satisfiers(this_obj, 'y', g)
  return options

To make sure it worked, you can see if you get `{'a'}` for this:

In [ ]:
print(find_options(obj))

### Define `do_imperative()` version 1

We still haven't defined the `do_imperative()` function (and so the robot
can't yet handle imperatives), but we now know better what it should look like.
It should find the VP, then the verb, and if the verb is "take", it should find
the possible objects it could be referring to.  The structure of the function
is like this:

In [ ]:
# version 1, handles only take
def do_imperative(parse):
  vp = parse[0][0][0]
  v = vp[0][0]
  if v == 'take':
    obj_opts = find_options(vp[1])
    if do_take(obj_opts):
      plot_world()
  else:
    print("I'm unsure what you are asking me to do.")

# version 1, finds object but then does not actually take
def do_take(obj_opts):
  if len(obj_opts) == 0:
    print('There is no such object to take.')
  else:
    print('out of order.')
  return False
    
type_handlers['imp'] = do_imperative

Of course, we want to replace the "out of order" message with some actual
action (by redefining `do_take()` later).  But at this point, `chat()` should at least run, and give you an
"out of order" message if you try to "take a red block", and a
"there is no such object to take" message if you try to "take a green block".

### ACTIVITY. Chat cell: take a green block

In [ ]:

chat()

### Support functions for `do_take()`

Now, let's consider all the possible scenarios for when someone tells the robot to take something.

If there are several options (individuals that meet the description), the robot needs to pick one.  If the robot already has
something in its hand, it might be what the robot needs, in which case it
doesn't need to do anything.  But if it has something different
in its hand, it needs to put that down first, somewhere.
And it can't pick something up that's underneath something else.

Things get surprisingly complicated once we start involving the world.

Since the world is designed in such a way that it has more squares
than other objects, the robot is guaranteed always to have an empty square available on which to put any object it is holding.
We can define a function
that will locate an empty square, and be confident that it'll find one.

In [ ]:
def empty_square():
  for square in squares:
    if not whats_on(square):
      return square

If you execute this function, it should find an empty square.
In fact, it should find `s3` at present.  This is a place where the robot could put whatever is in its hand.

In [ ]:
empty_square()

If we are not putting something down on a floor square but on another object, we can only put things down on things that are not under something else.  We can define `visible_things()` to filter a list of objects to just those that are visible.  Then any of these are potential landing sites for putting something down.  Constructed as a set because order is not relevant.

In [ ]:
def visible_things(objects):
  return {obj for obj in objects if not whats_on(obj)}

In [ ]:
# presently visible things
print(visible_things(dom))

Visible things are candidates for putting things down on, but the set of things the robot can pick up is more limited.  Specifically, it cannot pick up floor squares.  So the set of things that are possible things to pick up is the set of visible things but without any of the floor squares.


### TASK 6. Define `takable_things()`

Define a function called `takable_things()` that will take a set of objects anre return only those that are both visible nad not floor squares.

In [ ]:
# Task 6. Define takable_things() as visible things but excluding floor squares
# fill in the last line to return the right set
def takable_things(objects):
  return # put something here that gives the right answer

If this worked, the list of takable things should have just 'a', 'b', 'c', and 'e' in it.

In [ ]:
print(takable_things(dom))
# check to make sure it returns what we expect (assumes world in initial state)
assert takable_things(dom) == {'a', 'b', 'c', 'e'}

Now, to put an object down, the robot will put the object on another
object.  This means that we remove the object from the robot's hand, and we put
it in an `on` relation to another object.  So, here is a function that makes
this happen.

> I used a kind of nifty little construct above in the last line there below.
The `|=` operator is kind of like `+=`, it's a shorthand.
So the last line is equivalent to
> ```python
    val['on'] = val['on'] | {(obj_to_place, target)}
```

In [ ]:
def put_on(target):
  obj_to_place = obj_in_hand()
  val['held'] = {}
  val['on'] |= {(obj_to_place, target)}

Since both `val['on']` and `{(obj_to_place, target)}` are sets,
the `|` operator performs a set union, meaning that it adds all of members
together and puts them in `val['on']`.  Effectively, it is adding one more
"on" relation to the model.

Lastly, to pick an object up, we put it in the robot's hand, and remove
the `on` relation that previously indicated that it was on top of something.

In [ ]:
def pick_up(obj):
  val['held'] = {(obj,)}
  val['on'] = {(x,y) for (x,y) in val['on'] if x != obj}

### Assembling `do_take()`

Those are all the pieces we need to implement *take*.

Describing the function in English is probably not much clearer than
just reading the code below that redefines `do_take()`.  But:

The new function gets a list of options as an argument (that would have already been computed by evaluating the object, back in `do_imperative()`).

If there are no options, it prints a
message saying so.  (And this much we already had in the skeletal
version of `do_take()` we defined above.)  The function will default
to returning `False` (indicating the world did not change and
we do not need to redraw it), but if something happens, we will
return `True` when it does.

If there are options but one of those options is already in the hand,
print a message saying so and report success (`True`).
Otherwise, narrow the options to the takable objects (those that
are not squares and not underneath something).  If that leaves no
options, print a message saying so.  Otherwise, put the thing
currently in hand down on an empty square, pick up the first of the viable
options, and report success (`True`).  If we haven't already reported
success, then report failure (`False`).

In [ ]:
# version 2, this one actually takes
def do_take(obj_opts):
  if len(obj_opts) == 0:
    print('There is no such object to take.')
  else:
    if obj_in_hand() in obj_opts:
      print('I am already holding such a thing.')
      return True
    else:
      takable = takable_things(obj_opts)
      if len(takable) == 0:
        print('I cannot see such a thing to take.')
      else:
        put_on(empty_square())
        # take the first option
        obj_to_take = list(takable)[0]
        pick_up(obj_to_take)
        print('Object taken.')
        return True
  return False

### ACTIVITY. Chat cell: Take some stuff

Play around with it a little by running `chat()`. 
It's kind of fun.  Look.  Take a block.
Look. Take a red block.  Maybe it's not all that fun.  But it's a little bit fun.

In [ ]:
chat()

## Putting things down

The next thing to implement is *put*, but we already have a lot of those
pieces in place just from *take*.  In order to put something somewhere, the
robot must take it first and then put it in the target location.  However, now the
target need not be a floor square, meaning that we can make stacks of things.

Since the robot might not already be holding anything when we ask it to put something somewhere, the first step is to *take* the object we are supposed to
be putting somewhere.

We also need to determine whether we can take
the object (if it is not a square and not hidden), and then we need to determine whether the place we want to put
it is visible.  And then we do it.  The following function does this.

In [ ]:
def do_put(obj_opts, vp):
  if do_take(obj_opts):
    pp = vp[2]
    loc_opts = find_options(pp[1])
    if len(loc_opts) == 0:
      print('There is no such place to put anything.')
    else:
      visible = visible_things(loc_opts) - {obj_in_hand()}
      if len(visible) == 0:
        print("I cannot see such a place to put anything.")
      else:
        # pick the first option
        target = list(visible)[0]
        put_on(target)
        print('Object placed.')
        return True
  return False

### TASK 7. Add `do_put()` to `do_imperative()`

Redefine `do_imperative()` so that if will call the `do_put(obj_opts)` function if the verb is `'put'`.
You should be using what it does when the verb is `'take'` as a model, it is not very complicated.  But it does
require that you're kind of following along.

>The first version of `do_imperative()` (that we are now about to replace) was defined at the top of the "Taking things" section. Model the new definition on that one, but add a check for the verb "put" and then call `do_put()` with the correct arguments if the verb is "put".  Like with "take", if `do_put()` returns `True` then do `plot_world()` to show the updated state.

In [ ]:
# Answer 7.  Redefine do_imperative(parse) to include do_put(obj_opts)
# Fill in the blank lines with the part that handles the case where the verb is "put"
def do_imperative(parse):
  vp = parse[0][0][0]
  v = vp[0][0]
  if v == 'take':
    obj_opts = find_options(vp[1])
    if do_take(obj_opts):
      plot_world()
  # put the code that handles 'put' here



  else:
    print("I'm unsure what you are asking me to do.")

When we redefine `do_imperative` we need to add it to `type_handlers` again because its identity has changed.  It's now a new function, and so we need to point `type_handlers['imp']` to our new improved one.

In [ ]:
# because we redefined do_imperative, we need to update it in type_handlers
type_handlers['imp'] = do_imperative

Now you should be able to run `chat()` and start putting things on
other things.  Like "put a red block on a pyramid".


### ACTIVITY. Putting stuff down

In [ ]:
chat()

At this point, since you have been changing the world around, you might want to
be able to just reset it to its initial state.  If you find you want to
do that, you can define and use this:

In [ ]:
# reset the world to its original initial state
def reset_world():
  val['on'] = {('a','s1'),('b','s2'),('d','s4'),('c','d')}
  val['held'] = {('e',)}

In [ ]:
reset_world()

## Further directions

This is already very long, so we won't do much else here.
But there a bunch of things that would be neat to add, and are kind of
within reach.  If you're interested to keep going and try some of these, feel free to give them a whirl.  We are going to implement one addition (making pyramids be ineligible to put things on), but here are some others:

 - the real SHRDLU had a planning module, so that if you
said you wanted to pick up something that was covered, it would move
things out of the way so it could get it.  That would be relatively easy to
implement.

 - We could also add *wh*-questions, to handle things like
"what is on the blue pyramid" or "how many blocks are on even squares" or
"where is the blue pyramid".

 - Our support for the universal quantifier is
a bit incomplete as well; you can ask "is every block on an even square"
but you can't "put every pyramid on a block".  This could be shored up,
though it's actually a bit of a harder problem.

 - SHRDLU also had the ability
to learn things like that a stack of a block and pyramid can be called a
"steeple" (and then "steeples" can be referred to from then on), and could
absorb facts like "I like red blocks" and then answer questions about them.
Some kind of memory of names like this could be added.

 - Another easy-ish fix we could add is to deal with the fact that our robot
only knows singulars, not plurals.

 - We could add handling for
definite noun phrases.  At the moment, our robot treats "the pyramid" and
"a pyramid" exactly the same way.
But if we ask the robot to do something with "the block" it should be unsure
which block we meant.  Unless we had just been interacting with a block
recently, in which case we can assume that it was that block we meant.  Some kind of discourse memory would allow distinguishing those situations.

 - We could enhance the world with more objects, a third dimension, better graphics.

 - We could add more prepositions (like "under", "beside")

## Pyramids are pointy ##

Here is just one addition for you to do, without much explicit guidance.
The task is to prevent the robot from putting anything on a pyramid.

You can just make this say "I can't put that there" or something if
an attempt is to put something on a pyrmid.  That would be fine.  You could even try to make it so sophisticated that if you put something on a pyramid, it falls off onto a neighboring object, though this starts getting very complicated quickly.  (What if there is no adjacent column it could fall into because both adjacent columns have taller stacks of objects? What if you later move one of the horizontal supports?)  All you really need to do is make it refuse to put anything on a pyramid.

An obvious way to implement this is to alter `do_put()` so that if you give it a pyramid target, the robot will not honor it.  Possibly it could be considered not to be a visible (eligible) surface, for example.  There are various ways you can implement this.

### TASK 8. Pyramids are pointy

Modify `do_put` so that if prevents you from putting anything
on a pyramid.

In [ ]:
# Task 8: Modify the robot so that it prevents you from putting anything on a pyramid

